### 学習中断・再開の実装 --
* train, validともにtrain.csv全データを使って、
    * 一気に5epoch
    * 1epochずつ5epoch <br>
    行ったときの結果が一致するかを確認する --

In [1]:
import pandas as pd
import numpy as np

import torch
from transformers import AdamW

from bert_utils import *
from config import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

****** SEED fixed : 42 ******




In [2]:
train_df = pd.read_csv(data_path+"train.csv")

In [3]:
train_df["clean_text"] = train_df["text"].map(lambda x: clean_text(x))

In [4]:
tokenizer = define_tokenizer("cl-tohoku/bert-base-japanese")

In [5]:
train_ds = HateSpeechDataset(train_df, tokenizer=tokenizer, max_length=76, num_classes=2, text_col="clean_text", label_name="label")
valid_ds = HateSpeechDataset(train_df, tokenizer=tokenizer, max_length=76, num_classes=2, text_col="clean_text", label_name="label")

In [6]:
train_loader = DataLoader(
    train_ds, batch_size=32, num_workers=2, shuffle=True, pin_memory=True, drop_last=True
)
valid_loader = DataLoader(
    valid_ds, batch_size=64, num_workers=2, shuffle=False, pin_memory=True
)

In [7]:
model = HateSpeechModel("cl-tohoku/bert-base-japanese", 2, custom_header="concatenate-4", dropout=0.2, n_msd=None)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


concatenate-4


In [8]:
# Define Optimizer and Scheduler --
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-6)
scheduler = fetch_scheduler(optimizer=optimizer, scheduler="None")

*** *** NOT implemented *** *** 
        --> CosineAnnealingLR *** *** 


### 初期状態から5epoch回す --

In [9]:
model.to(device)

model, history = run_training(
    model, train_loader, valid_loader,
    optimizer, scheduler, 1, device,
    True, 5, "_ALL", "./test/",
    log=None, save_checkpoint=True, load_checkpoint=None
) # -> "./test/checkpoint-fold_ALL.pth"

[INFO] Using GPU : NVIDIA GeForce RTX 3090



100%|██████████| 83/83 [00:05<00:00, 15.42it/s, Epoch=1, LR=7.6e-6, Valid_Loss=0.123]


Valid Loss Improved : inf ---> 0.122735
Model Saved



 90%|████████▉ | 147/164 [00:09<00:01, 15.32it/s, Epoch=2, LR=3.2e-6, Train_Loss=0.113] 


KeyboardInterrupt: 